# Combine Area_1 fivemin power data

Read fivemin power CSVs from each month folder (Area_1), combine into one dataset, and add:
- **year**, **month**, **day** (from datetime)
- **block**: eu_1–8 → 1, eu_9–16 → 2, eu_17–24 → 3, eu_25–36 → none
- **crop**: per-eu mapping (strawberry, broccoli, pepper, etc.)

In [1]:
import pandas as pd
import os
from pathlib import Path

# Month folders (apr_2025, ...) are in parent of all_data (Solar)
BASE = Path.cwd().parent if Path.cwd().name == 'all_data' else Path.cwd()
# Paths: {folder_name: (folder_path, fivemin_csv_filename)}
MONTH_CONFIG = [
    ('apr_2025', 'april_fivemin_power.csv'),
    ('may_2025', 'may_fivemin_power.csv'),
    ('jun_2025', 'june_fivemin_power.csv'),
    ('jul_2025', 'july_fivemin_power.csv'),
    ('aug_2025', 'august_fivemin_power.csv'),
    ('sep_2025', 'september_fivemin_power.csv'),
    ('oct_2025', 'october_fivemin_power.csv'),
    ('nov_2025', 'november_fivemin_power.csv'),
    ('dec_2025', 'december_fivemin_power.csv'),
    ('jan_2026', 'january_fivemin_power.csv'),
]

In [2]:
# Load and combine all Area_1 fivemin power CSVs
dfs = []
for folder, fname in MONTH_CONFIG:
    path = BASE / folder / 'Area_1' / fname
    if not path.exists():
        print(f"Skip (not found): {path}")
        continue
    df = pd.read_csv(path)
    # Normalize datetime column name (some files may use "time (min)")
    dt_col = [c for c in df.columns if 'time' in c.lower() or c == 'datetime'][0]
    df = df.rename(columns={dt_col: 'datetime'})
    df['datetime'] = pd.to_datetime(df['datetime'])
    dfs.append(df)
    print(f"Loaded {path.name}: {len(df)} rows")

combined = pd.concat(dfs, ignore_index=True)
combined = combined.sort_values('datetime').reset_index(drop=True)
print(f"\nCombined: {len(combined)} rows, {combined['datetime'].min()} to {combined['datetime'].max()}")

Loaded april_fivemin_power.csv: 3744 rows
Loaded may_fivemin_power.csv: 8928 rows
Loaded june_fivemin_power.csv: 8640 rows
Loaded july_fivemin_power.csv: 8928 rows
Loaded august_fivemin_power.csv: 8928 rows
Loaded september_fivemin_power.csv: 8640 rows
Loaded october_fivemin_power.csv: 8928 rows
Loaded november_fivemin_power.csv: 8652 rows
Loaded december_fivemin_power.csv: 8928 rows
Loaded january_fivemin_power.csv: 8928 rows

Combined: 83244 rows, 2025-04-18 00:00:00 to 2026-01-31 23:55:00


In [3]:
# Extract year, month, day from datetime
combined['year'] = combined['datetime'].dt.year
combined['month'] = combined['datetime'].dt.month
combined['day'] = combined['datetime'].dt.day

In [4]:
# Block: eu_1-8 → 1, eu_9-16 → 2, eu_17-24 → 3, eu_25-36 → none
def block_for_eu(eu_num):
    if 1 <= eu_num <= 8: return 1
    if 9 <= eu_num <= 16: return 2
    if 17 <= eu_num <= 24: return 3
    return None  # 25-36

# Crop mapping (eu_25 to eu_36 are none)
CROP_MAP = {
    1: 'strawberry', 11: 'strawberry', 24: 'strawberry',
    2: 'broccoli', 9: 'broccoli', 22: 'broccoli',
    3: 'pepper', 14: 'pepper', 19: 'pepper',
    4: 'Polli mix 1', 12: 'Polli mix 1', 23: 'Polli mix 1',
    5: 'grass', 15: 'grass', 17: 'grass',
    6: 'Squash', 10: 'Squash', 18: 'Squash',
    7: 'Polli mix 2', 13: 'Polli mix 2', 21: 'Polli mix 2',
    8: 'Raspberry', 16: 'Raspberry', 20: 'Raspberry',
}
def crop_for_eu(eu_num):
    return CROP_MAP.get(eu_num, 'none')

In [7]:
# Reshape to long format: one row per (datetime, eu)
id_vars = ['datetime', 'year', 'month', 'day']
eu_cols = [c for c in combined.columns if c.startswith('eu_') and ' (W)' in c]
long = combined[id_vars + eu_cols].melt(
    id_vars=id_vars,
    value_vars=eu_cols,
    var_name='eu (W)',
    value_name='power_W'
)
# Values: eu_1, eu_2, ... (strip " (W)" from original "eu_1 (W)")
long['eu (W)'] = long['eu (W)'].str.replace(r'\s*\(W\)\s*', '', regex=True)
# Use eu number only for block/crop mapping (no eu_num column kept)
eu_num = long['eu (W)'].str.extract(r'eu_(\d+)')[0].astype(int)
long['block'] = eu_num.map(block_for_eu).astype('Int64')  # 1, 2, 3 (not 1.0, 2.0); NA for none
long['crop'] = eu_num.map(crop_for_eu)
long.head(10)

,datetime,year,month,day,eu (W),power_W,block,crop
0,2025-04-18 00:00:00,2025,4,18,eu_1,0.0,1,strawberry
1,2025-04-18 00:05:00,2025,4,18,eu_1,0.0,1,strawberry
2,2025-04-18 00:10:00,2025,4,18,eu_1,0.0,1,strawberry
3,2025-04-18 00:15:00,2025,4,18,eu_1,0.0,1,strawberry
4,2025-04-18 00:20:00,2025,4,18,eu_1,0.0,1,strawberry
5,2025-04-18 00:25:00,2025,4,18,eu_1,0.0,1,strawberry
6,2025-04-18 00:30:00,2025,4,18,eu_1,0.0,1,strawberry
7,2025-04-18 00:35:00,2025,4,18,eu_1,0.0,1,strawberry
8,2025-04-18 00:40:00,2025,4,18,eu_1,0.0,1,strawberry
9,2025-04-18 00:45:00,2025,4,18,eu_1,0.0,1,strawberry


In [9]:
# Save combined long-format file (with year, month, day, block, crop)
out_path = Path.cwd() / 'area1_fivemin_power_combined.csv'
long.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
print(f"Shape: {long.shape}")
long.dtypes

Saved: c:\Users\nhphuong\Desktop\Solar\all_data\area1_fivemin_power_combined.csv
Shape: (2996784, 8)


datetime    datetime64[ns]
year                 int32
month                int32
day                  int32
eu (W)              object
power_W            float64
block                Int64
crop                object
dtype: object

In [8]:
# Quick check: block and crop value counts
print("Block counts:")
print(long['block'].value_counts(dropna=False))
print("\nCrop counts:")
print(long['crop'].value_counts())

Block counts:
block
<NA>    998928
1       665952
2       665952
3       665952
Name: count, dtype: Int64

Crop counts:
crop
none           998928
strawberry     249732
broccoli       249732
Polli mix 1    249732
pepper         249732
grass          249732
Squash         249732
Polli mix 2    249732
Raspberry      249732
Name: count, dtype: int64
